In [18]:
import pandas as pd
import numpy as np

In [65]:
def softmax(x):
    
    f_x = np.exp(x) / np.sum(np.exp(x))
    return f_x
# def softmax(x):

#     y = np.exp(x - np.max(x))
#     f_x = y / np.sum(np.exp(x))
#     return f_x
def normalize(df):
    # return (df-df.min())/(df.max()-df.min())
    return df / np.linalg.norm(df)

In [97]:
ppl_kb = pd.read_csv('/nas/home/gujiashe/trans/yago310_ppls_top10_kb.tsv', sep='\t', index_col=0)
ppl_kb = ppl_kb[ppl_kb["rank"]<=10]
ppl_kb.index = list(range(len(ppl_kb)))
ppl_kb.shape

(31100, 7)

In [135]:
ppl_gpt = pd.read_csv('/nas/home/gujiashe/trans/yago310_ppls_top10_gpt.tsv', sep='\t', index_col=0)
ppl_gpt = ppl_gpt[ppl_gpt["rank"]<=10]
ppl_gpt.index = list(range(len(ppl_gpt)))
ppl_gpt.shape

(31100, 7)

In [183]:
def normalize_score_ppl(ppl):
    ppl_norm = ppl.copy()
    for i in range(len(ppl)//10):
        ppl_norm.loc[i*10: i*10+9, "score"] = normalize(softmax(ppl_norm.loc[i*10: i*10+9, "score"]))
        ppl_norm.loc[i*10: i*10+9, "ppls"] = normalize(ppl_norm.loc[i*10: i*10+9, "ppls"])
    return ppl_norm
ppl_norm = normalize_score_ppl(ppl_gpt)

In [184]:
def multiply_score_ppl(ppl_norm):
    ppl_multiply = ppl_norm.copy()
    for i in range(len(ppl_norm)//10):
        ppl_multiply.loc[i*10: i*10+9, "multiply"] = ppl_multiply.loc[i*10: i*10+9, "score"] * 1/ppl_multiply.loc[i*10: i*10+10, "ppls"]
        ppl_multiply.loc[i*10: i*10+9, "multiply_rank"] = ppl_multiply.loc[i*10: i*10+9, "multiply"].rank()
        ppl_multiply.loc[i*10: i*10+9, "multiply_rank_true"] = ppl_multiply.loc[i*10-1 + ppl_multiply.loc[i*10, 'rank'], "multiply_rank"]  
    return ppl_multiply
ppl_multiply = multiply_score_ppl(ppl_norm)

In [185]:
def plus_score_ppl(ppl_norm):
    ppl_plus = ppl_norm.copy()
    for i in range(len(ppl_norm)//10):
        ppl_plus.loc[i*10: i*10+9, "plus"] = ppl_plus.loc[i*10: i*10+9, "score"] + 1/ppl_plus.loc[i*10: i*10+10, "ppls"]
        ppl_plus.loc[i*10: i*10+9, "plus_rank"] = ppl_plus.loc[i*10: i*10+9, "plus"].rank()
        ppl_plus.loc[i*10: i*10+9, "plus_rank_true"] = ppl_plus.loc[i*10-1 + ppl_plus.loc[i*10, 'rank'], "plus_rank"]  
    return ppl_plus
ppl_plus = plus_score_ppl(ppl_norm)

In [186]:
def print_rank(rank, k):
    print((rank <= k).value_counts(True)[True])
for i in [1, 5]:
    print("hits@{}".format(i))
    print_rank(ppl_gpt['rank'], i)
    print_rank(ppl_multiply['multiply_rank_true'], i)
    print_rank(ppl_plus['plus_rank_true'], i)

In [195]:
kb = pd.read_csv('/nas/home/gujiashe/critic/lp.logits.txt', sep='\t', header=None)

In [199]:
for i in range(len(kb)):
    kb.loc[i] = normalize(softmax(kb.loc[i]))
 

In [206]:
top10 = pd.read_csv('/nas/home/gujiashe/trans/yago310_top10_predictions.tsv', sep='\t', index_col=0)
top10 = top10[top10["rank"]<=10]
top10.index = list(range(len(top10)))
top10.shape

(31100, 6)

In [207]:
def normalize_score(top10):
    top10_norm = top10.copy()
    for i in range(len(top10)//10):
        top10_norm.loc[i*10: i*10+9, "score"] = normalize(softmax(top10_norm.loc[i*10: i*10+9, "score"]))
    return top10_norm
top10_norm = normalize_score(top10)

In [217]:
top10_kb = top10_norm.copy()
top10_kb['kb'] = kb[1]

In [226]:
def multiply_score_kb(top10_kb):
    top10_kb_multiply = top10_kb.copy()
    for i in range(len(kb)//10):
        top10_kb_multiply.loc[i*10: i*10+9, "multiply"] = top10_kb_multiply.loc[i*10: i*10+9, "score"] * top10_kb_multiply.loc[i*10: i*10+10, "kb"]
        top10_kb_multiply.loc[i*10: i*10+9, "multiply_rank"] = top10_kb_multiply.loc[i*10: i*10+9, "multiply"].rank()
        top10_kb_multiply.loc[i*10: i*10+9, "multiply_rank_true"] = top10_kb_multiply.loc[i*10-1 + top10_kb_multiply.loc[i*10, 'rank'], "multiply_rank"]  
    return top10_kb_multiply
top10_kb_multiply = multiply_score_kb(top10_kb)

In [227]:
def plus_score_kb(top10_kb):
    top10_kb_plus = top10_kb.copy()
    for i in range(len(kb)//10):
        top10_kb_plus.loc[i*10: i*10+9, "plus"] = top10_kb_plus.loc[i*10: i*10+9, "score"] + top10_kb_plus.loc[i*10: i*10+10, "kb"]
        top10_kb_plus.loc[i*10: i*10+9, "plus_rank"] = top10_kb_plus.loc[i*10: i*10+9, "plus"].rank()
        top10_kb_plus.loc[i*10: i*10+9, "plus_rank_true"] = top10_kb_plus.loc[i*10-1 + top10_kb_plus.loc[i*10, 'rank'], "plus_rank"]  
    return top10_kb_plus
top10_kb_plus = plus_score_kb(top10_kb)

In [229]:
def print_rank(rank, k):
    print((rank <= k).value_counts(True)[True])
for i in [1, 5]:
    print("hits@{}".format(i))
    print_rank(top10['rank'], i)
    print_rank(top10_kb_multiply['multiply_rank_true'], i)
    print_rank(top10_kb_plus['plus_rank_true'], i)

hits@1
0.14083601286173633
0.01864951768488746
0.01929260450160772
hits@5
0.4585209003215434
0.2237942122186495
0.21286173633440514
